### Python & Banco de Dados

O passo a passo é para Windows. Caso tenha outro sistema operacional, o objetivo é ter o python e banco de dados na sua máquina <br><br>

1 - Instalar o Anaconda para ter uma plataforma Python Local: https://www.anaconda.com/<br>
2 - Não é necessário marcar nenhuma opção durante a instalação<br>
3 - Ao terminar a instalação, abrir o Jupyter Notebook<br>
4 - Executar o comando pip install psycopg2. Mais orientações em https://pypi.org/project/psycopg2/


In [4]:
pip install psycopg2


5 - Considerando o banco de dados utilizado desde a primeira aula (dvdrental) faça a conexão abaixo

In [6]:
import psycopg2
#conn = psycopg2.connect("dbname=dvdrental user=postgres password=admin")

# ou pode ser feito como abaixo:
conn = psycopg2.connect(
    host="localhost", 
    database="dvdrental",
    user="postgres",
    password="admin")

Passo para a conectar

1 - Abrir a conexao (commando connect)
2 - Abrir o cursor
3 - Executar o comando
4 - Fechar o cursor
5 - fechar a conexao

In [9]:
# 1 - Abrir a conexao
conn = psycopg2.connect("dbname=dvdrental user=postgres password=admin")
    
cursor = conn.cursor()
    
cursor.execute('SELECT version()')
    
db_version = cursor.fetchone()
print(db_version)

('PostgreSQL 14.4, compiled by Visual C++ build 1914, 64-bit',)


In [10]:
# Mostrando a versão do banco de dados

try: # neste bloco ele ira tentar abrir a conexao e executar os comandos sql
    # 1 - Abrir a conexao
    conn = psycopg2.connect("dbname=dvdrental user=postgres password=admin")

    # 2 - Abrir o cursor
    cursor = conn.cursor()

    # 3 - Executar o comando
    print("A versão do banco:")
    cursor.execute('SELECT version()')

    # 3.5 -> usou o fetchone() (fetch one - pegar um) para pegar a primeira linha do resultado
    db_version = cursor.fetchone()
    print(db_version)

    # 4 - Fechar o cursor
    cursor.close()

except (Exception, psycopg2.DatabaseError) as error: # neste bloco, caso de algum erro no bloco anterior,
                                                     #ele ira mostrar o erro
    print(error)
finally: # independente se o bloco try deu erro ou nao, ele ira fazer esta parte, que e tentar fechar a conexao
    if conn is not None:
        # 5 - fechar a conexao
        conn.close()
        print('Conexão encerrada.')


A versão do banco:
syntax error at or near "SELCT"
LINE 1: SELCT version()
        ^

Conexão encerrada.


In [21]:
import pandas as pd

# 1 - Abrir a conexao
conn = psycopg2.connect("dbname=dvdrental user=postgres password=admin") 

# 2 - Abrindo um cursor

cursor = conn.cursor()


select_query = 'select language_id, name, last_update from language'


try:
    # 3- Executando o meu comando
    cursor.execute(select_query)
    
    # 3.5 - Fazendo alguma coisa com o meu comando
    resultado = cursor.fetchall()  # pegou todas as linhas do resultado (fetch ALL)
    
    # 4 - Fechou o cursor
    cursor.close()
    
except (Exception, psycopg2.DatabaseError) as error:
        print("Erro: %s" % error)
        cursor.close()
finally:
    if conn is not None:
        conn.close()
        
print(resultado[1])

#resultado.info()
#df = pd.DataFrame(resultado, columns=['language_id', 'name', 'last_update'])

#print(df)

(2, 'Italian             ', datetime.datetime(2006, 2, 15, 10, 2, 19))


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   language_id  6 non-null      int64         
 1   name         6 non-null      object        
 2   last_update  6 non-null      datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 272.0+ bytes


In [37]:
def insert_actor(_actor_id, _first_name, _last_name):
    
    """ Insere um novo ator na tabela de atores """
    
    #sql = f"""INSERT INTO actor (actor_id, first_name, last_name)
    #         VALUES(%s,%s,%s) ;"""
    
    sql = f"""INSERT INTO actor (actor_id, first_name, last_name)
             VALUES(%s,%s,%s) ;"""
    #sql = f"INSERT INTO actor (actor_id, first_name, last_name) VALUES({_actor_id},'{_first_name}','{_last_name}') ;"
    conn = None
    
    #INSERT INTO NOME_DA_TABELA (COLUNA1, COLUNA2, ...) VALUES (VALOR_DA_COLUNA_1, VALOR_DA_COLUNA_2, ...)
    
    try:
        # connect to the PostgreSQL database
        conn = psycopg2.connect("dbname=dvdrental user=postgres password=admin")
        # create a new cursor
        cur = conn.cursor()
        # execute the INSERT statement
        cur.execute(sql, (_actor_id,_first_name, _last_name,))
        #cur.execute(sql)
        # get the generated id back
        #vendor_id = cur.fetchone()[0]
        # estou confirmando as alteracoes no banco
        conn.commit()
        
        # fechar o cursor
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

    return _actor_id

In [24]:
insert_actor(1001,"Adriano","Goncalves")

1001

In [30]:
insert_actor("Teodor", "Barczewski")

In [32]:
def max_actor():
    
    sql = "select max(actor_id) from actor"
    actor_id = None
    
    try:
        conn = psycopg2.connect("dbname=dvdrental user=postgres password=admin")
        # create a new cursor
        cur = conn.cursor(sql)
        cur.execute(sql)
        
        actor_id = cur.fetchone()
        
        cur.close()
        
    except(Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
     
    
    return actor_id 

In [33]:
max_actor()[0]

1001

In [38]:
insert_actor(max_actor()[0]+1,"Mateus","Alves")

1002

In [65]:
# Import pandas
import pandas as pd

# Create dataframe
data = pd.DataFrame({
    'book_id':[12345, 12346, 12347,38383],
    'title':['Python Programming', 'Learn MySQL', 'Data Science Cookbook','Crepusculo'],
    'price':[29, 23, 27,69],
    'quantity':[2,5,3,10]
})

data

,book_id,title,price,quantity
0,12345,Python Programming,29,2
1,12346,Learn MySQL,23,5
2,12347,Data Science Cookbook,27,3
3,38383,Crepusculo,69,10


In [66]:
data['ValorTotal'] = data['price']*data['quantity']
data

,book_id,title,price,quantity,ValorTotal
0,12345,Python Programming,29,2,58
1,12346,Learn MySQL,23,5,115
2,12347,Data Science Cookbook,27,3,81
3,38383,Crepusculo,69,10,690


In [42]:
cur.close()
conn.close()

NameError: name 'cur' is not defined

In [61]:
conn = psycopg2.connect("dbname=dvdrental user=postgres password=admin")
cur = conn.cursor()

print("Criando um banco")

try:
    cur.execute("CREATE TABLE IF NOT EXISTS book_details(book_id int,title varchar(50),price int, quantity int, valortotal int, last_update timestamp without time zone NOT NULL DEFAULT now())")   
    
    conn.commit() #alteracoes no banco precisam deste comando para formalizar a execucao
    # bom evitar a conexão aberta.
    cur.close()

except (Exception, psycopg2.DatabaseError) as error:
    print(error)
finally:
    if conn is not None:
        conn.close()
        print('Conexão encerrada.')

Criando um banco
Conexão encerrada.


In [64]:
data

,book_id,title,price,quantidade,ValorTotal
0,12345,Python Programming,29,2,58
1,12346,Learn MySQL,23,5,115
2,12347,Data Science Cookbook,27,3,81
3,38383,Crepusculo,69,10,690


In [53]:
#data.rename(columns={'quantity': 'quantidade'}, inplace=True)

In [54]:
data

,book_id,title,price,quantidade,ValorTotal
0,12345,Python Programming,29,2,58
1,12346,Learn MySQL,23,5,115
2,12347,Data Science Cookbook,27,3,81
3,38383,Crepusculo,69,10,690


In [69]:
conn = psycopg2.connect("dbname=dvdrental user=postgres password=admin")
cursor = conn.cursor()

# criando uma lista para inclusao
cols = ",".join([str(i) for i in data.columns.tolist()])
print(cols)

# Insert DataFrame recrds one by one.
for i,row in data.iterrows():
    sql = "INSERT INTO book_details (" +cols + ") VALUES (" + "%s,"*(len(row)-1) + "%s)"
    print(sql)
    cursor.execute(sql, tuple(row))
    print(tuple(row))

    # the connection is not autocommitted by default, so we must commit to save our changes
    conn.commit()


book_id,title,price,quantity,ValorTotal
INSERT INTO book_details (book_id,title,price,quantity,ValorTotal) VALUES (%s,%s,%s,%s,%s)
(12345, 'Python Programming', 29, 2, 58)
INSERT INTO book_details (book_id,title,price,quantity,ValorTotal) VALUES (%s,%s,%s,%s,%s)
(12346, 'Learn MySQL', 23, 5, 115)
INSERT INTO book_details (book_id,title,price,quantity,ValorTotal) VALUES (%s,%s,%s,%s,%s)
(12347, 'Data Science Cookbook', 27, 3, 81)
INSERT INTO book_details (book_id,title,price,quantity,ValorTotal) VALUES (%s,%s,%s,%s,%s)
(38383, 'Crepusculo', 69, 10, 690)


In [56]:
conn = psycopg2.connect("dbname=dvdrental user=postgres password=admin")
cursor = conn.cursor()
# Execute query
sql = "SELECT * FROM book_details"
cursor.execute(sql)

# Fetch all the records
result = cursor.fetchall()
for i in result:
    print(i)

(12345, 'Python Programming', 29, 2, 58)
(12346, 'Learn MySQL', 23, 5, 115)
(12347, 'Data Science Cookbook', 27, 3, 81)
(38383, 'Crepusculo', 69, 10, 690)


In [57]:
conn.close()

Maiores detalhes em https://www.dataquest.io/blog/sql-insert-tutorial/